In [1]:
import theano

get_ipython().magic(u"cd '/home/hartmank/braindecode/convvisual'")
import numpy as np
get_ipython().magic(u'load_ext autoreload')
get_ipython().magic(u'autoreload 2')
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from matplotlib import cm

import matplotlib.lines as mlines
import seaborn
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
matplotlib.rcParams['font.size'] = 7
seaborn.set_style('darkgrid')

from braindecode.scripts.print_results import ResultPrinter
from braindecode.csp.print_results import CSPResultPrinter
import logging
log = logging.getLogger()
log.setLevel("DEBUG")
from braindecode.scripts.train_experiments import setup_logging
setup_logging()

from braindecode.experiments.load import load_exp_and_model
from braindecode.veganlasagne.layer_util import print_layers, recompute_bnorm_layer_statistics
import lasagne

Using gpu device 0: GeForce GTX 780 (CNMeM is disabled, cuDNN 5005)


/home/hartmank/braindecode/convvisual


/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
savepath = 'models/paper/ours/cnt/deep4/243/50BestInputsFor5BestFilters/'
exp, model = load_exp_and_model('../data/models/paper/ours/cnt/deep4/243', set_invalid_to_NaN=False)
get_ipython().magic(u'cd ../')

2016-12-19 17:14:22,619 Setting n_sample preds automatically to 479
2016-12-19 17:14:22,620 Input window length is 522
2016-12-19 17:14:22,694 Setting n_sample preds automatically to 479
2016-12-19 17:14:22,695 Input window length is 522
/home/hartmank/braindecode


In [3]:
from braindecode.veganlasagne.layers import FinalReshapeLayer
print model
all_l = lasagne.layers.get_all_layers(model)
model = all_l[-3]
model = FinalReshapeLayer(model,remove_invalids=True,flatten=False)
model = lasagne.layers.DimshuffleLayer(model,(0,2,1,3))
model = lasagne.layers.ReshapeLayer(model,(-1,4))
model = lasagne.layers.NonlinearityLayer(model,nonlinearity=lasagne.nonlinearities.softmax)

In [4]:
print_layers(model)

 0-InputLayer                                                         [None, 44, 1000, 1]
 1-DimshuffleLayer                                                    (None, 1, 1000, 44)
 2-DropoutLayer             0.000                                    
 3-Conv2DLayer              10x1                                      (None, 25, 991, 44)
 4-Conv2DAllColsLayer       1x44                                      (None, 25, 991, 1)
 5-BatchNormLayer                                     elu            
 6-Pool2DLayer              3x1                       max             (None, 25, 989, 1)
 7-StrideReshapeLayer           ::3 ::1                               (None, 25, 330, 1)
 8-NonlinearityLayer                                                 
 9-DropoutLayer             0.500                                    
10-Conv2DLayer              10x1                                      (None, 50, 321, 1)
11-BatchNormLayer                                     elu            
12-Pool2DLayer          

In [5]:
exp.dataset.load()

2016-12-19 17:14:22,751 Load Training Set...
2016-12-19 17:14:41,358 Load Test Set...
2016-12-19 17:14:44,817 Clean Training Set...
2016-12-19 17:14:45,425 Rejected channels: []
2016-12-19 17:14:45,426 #Clean trials:     895
2016-12-19 17:14:45,426 #Rejected trials:  2
2016-12-19 17:14:45,427 Fraction Clean:    99.0%
2016-12-19 17:14:45,427 (from maxmin):     2
2016-12-19 17:14:45,428 (from var):        0
2016-12-19 17:14:45,595 Clean Test Set...
2016-12-19 17:14:45,703 Rejected channels: []
2016-12-19 17:14:45,704 #Clean trials:     160
2016-12-19 17:14:45,704 #Rejected trials:  0
2016-12-19 17:14:45,705 Fraction Clean:    100.0%
2016-12-19 17:14:45,705 (from maxmin):     0
2016-12-19 17:14:45,706 (from var):        0
2016-12-19 17:14:45,706 Create Cleaned Cnt Sets...
2016-12-19 17:14:46,820 Create sets from cleaned cnt...
2016-12-19 17:14:46,820 Preprocess continuous signal...
2016-12-19 17:15:06,263 Not doing any bandpass, since low 0 or None and high None
2016-12-19 17:15:20,237 Lo

In [6]:
#print 'loaded'
datasets = exp.dataset_provider.get_train_merged_valid_test(exp.dataset)
#print 'loaded2'
#inputs,targets = datasets['test'].get_data()
exp.iterator.batch_size = 100
test_batches = list(exp.iterator.get_batches(datasets['test'], shuffle=False))
inputs,targets = test_batches[0]

targets = targets.reshape((len(inputs),-1,4))
targets = targets.mean(axis=1)

In [7]:
#from braindecode.datahandling.batch_iteration import compute_trial_start_end_samples,CntWindowTrialIterator
#trial_iterator = CntWindowTrialIterator(1, 1145, 521,
#            check_preds_smaller_trial_len=True)
#dir(exp.iterator)
#exp.iterator.input_time_length
#len(compute_trial_start_end_samples(targets)[0])

In [8]:
#test_batches = list(exp.iterator.get_batches(datasets['test'], shuffle=False))
#len(test_batches)
#len(test_batches[5][0])

In [9]:
len(np.where(np.any(datasets['test'].get_data()[1]!=0,axis=1))[0])

100000

In [10]:
len(datasets['test'].get_data()[1])

308544

In [11]:
get_ipython().magic(u"cd '/home/hartmank/braindecode/convvisual'")
from convvisual import *
import os
import pickle

from braindecode.veganlasagne.layers import create_pred_fn
model = lasagne.layers.get_all_layers(model)

figpath = os.path.join('/home/hartmank/data/convvisual/figures',savepath)
datpath = os.path.join('/home/hartmank/data/convvisual/RF_data',savepath)
if not os.path.isdir(figpath):
    os.makedirs (figpath)
if not os.path.isdir(datpath):
    os.makedirs (datpath)

/home/hartmank/braindecode/convvisual


In [12]:
layer_list = [3,5,11,17,23,28]
X_reshapes = [None,None,None,None,None,81]
splits = [1,1,2,2,5,5]
n_Units = 50
n_filters = 5

layer_list = [28]
X_reshapes = [81]
splits = [5]

class_defs = [[1,0,0,0],
             [0,1,0,0],
             [0,0,1,0],
             [0,0,0,1]]

for layer,X_reshape,split in zip(layer_list,X_reshapes,splits):
    RF_model,RF_layer = check_if_finalreshape_is_needed(model,layer)
    #Get highest input/unit activation combination
    l_RF = receptive_field_build_deconv_layers(RF_model[RF_layer],RF_model[1],
                                                   use_learned_W=False,X_reshape=X_reshape)
    pred_fn = create_pred_fn(RF_model[RF_layer])
    output = pred_fn(list(inputs))
    
    c=0
    for class_def in class_defs:
        class_indeces = np.where(np.all(np.equal(targets,class_def),axis=1))[0]
        mean_filters = output[class_indeces].mean(axis=3).mean(axis=2).mean(axis=0)
        print 'Got filter indeces'
        print mean_filters.shape,mean_filters.argmax(axis=0)
        max_filters = mean_filters.argsort(axis=0)[::-1][:n_filters]
    
        max_ind = list()
        for filter in max_filters:
            print 'Filter',filter
            max_ind.append(get_most_active_units_in_layer_from_output(output,RF_layer,filter,n_units=n_Units,abs_act=False))
        #del output
        print 'Max filter finished'
        batches = np.array_split(np.arange(len(max_ind)),split)
        
        X_RF_complete = np.array([])
        max_ind = np.asarray(max_ind)
        for batch in batches:
            print 'Batch'
            if len(batch)==0:
                break

            max_ind_ = max_ind[batch]
            max_ind_shape = max_ind_.shape
            max_ind_ = max_ind_.reshape((1,-1,max_ind_shape[2])).squeeze()

            X_RF_complete_,mask = get_receptive_field_masked_inputs(inputs,max_ind_,l_RF)

            X_RF_shape = X_RF_complete_.shape
            X_RF_complete_ = X_RF_complete_.reshape(max_ind_shape[0],-1,X_RF_shape[1],X_RF_shape[2],X_RF_shape[3])
            X_RF_complete_.shape

            X_RF_complete = np.vstack([X_RF_complete, X_RF_complete_]) if X_RF_complete.size else X_RF_complete_

        pkl = open(os.path.join(datpath,'Class%d_Layer%02d.data'%(c,layer)), 'w')
        savedata = {'X':inputs,'Y':targets,'RF':X_RF_complete, 'Units':max_ind, 'filters':max_filters}
        pickle.dump(savedata,pkl)
        pkl.close()
        c+=1

/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode=self.mode,
/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode=self.mode,
/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode=self.mode,


Got filter indeces
(4,) 0
Filter 0
Filter 1
Filter 2
Filter 3
Max filter finished
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0


/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/lasagne/layers/helper.py:212: UserWarning: get_output() was called with unused kwargs:
	input_var
  % "\n\t".join(suggestions))


Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
Batch
Got filter indeces
(4,) 1
Filter 1
Filter 0
Filter 2
Filter 3
Max filter finished
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!

NameError: name 'data28' is not defined